In [1]:
from typing import Optional, List, Tuple
import pandas as pd
import numpy as np

from bank_schedule.data import Data
from bank_schedule.constants import RS, RAW_DATA_FOLDER

from bank_schedule.ortools_tsp import get_best_route
from bank_schedule.plot import geoplot_clusters
from bank_schedule.cluster import clusterize_atm_dichotomous, clusterize_atm, get_route_from_outlier

LABEL_COL = 'label'

In [2]:
loader = Data(RAW_DATA_FOLDER)
coords_df = loader.get_geo_TIDS()

In [3]:
dist_mtx = loader.get_distance_matrix()
geo_df = loader.get_geo_TIDS()
n_clusters = 5
allowed_percent = 5

size = int(np.ceil(geo_df.shape[0] / n_clusters))
clusters = {}
i = 0
while not dist_mtx.empty:
    clust = get_route_from_outlier(dist_mtx)[:size]
    clusters[i] = clust
    cond = dist_mtx['Origin_tid'].isin(clust) | dist_mtx['Destination_tid'].isin(clust)
    dist_mtx = dist_mtx[~cond].copy()
    i += 1

clust_df = coords_df.copy()
clust_df[LABEL_COL] = 0
for i, tids in clusters.items():
    clust_df.loc[clust_df['TID'].isin(tids), LABEL_COL] = i

geoplot_clusters(clust_df, LABEL_COL)

In [4]:
clust_df_dichotomous = clusterize_atm_dichotomous(loader, n_clusters=5, allowed_percent=4)

Обработка кластера   0, стартовый размер кластера: 1630
Расчетное время маршрута по кластеру после   0 итерации: 12166.55. Целевое время: 4933.62 - 5130.97 минут
Расчетное время маршрута по кластеру после   1 итерации:  6536.88. Целевое время: 4933.62 - 5130.97 минут
Расчетное время маршрута по кластеру после   2 итерации:  3481.52. Целевое время: 4933.62 - 5130.97 минут
Расчетное время маршрута по кластеру после   3 итерации:  5175.97. Целевое время: 4933.62 - 5130.97 минут
Расчетное время маршрута по кластеру после   4 итерации:  4325.39. Целевое время: 4933.62 - 5130.97 минут
Расчетное время маршрута по кластеру после   5 итерации:  4741.54. Целевое время: 4933.62 - 5130.97 минут
Расчетное время маршрута по кластеру после   6 итерации:  4925.98. Целевое время: 4933.62 - 5130.97 минут
Расчетное время маршрута по кластеру после   7 итерации:   4984.3. Целевое время: 4933.62 - 5130.97 минут
Обработка кластера   1, стартовый размер кластера: 1333
Расчетное время маршрута по кластеру пос

/Users/affernus/PROJECTS/hacks/bank_schedule/src/bank_schedule/cluster.py:274: UserWarning:

Решение не сошлось, получилось 6 кластеров (лишние точки будут добавлены в кластер 4). Попробуйте увеличить allowed_percent



In [5]:
clust_df_kmeans = clusterize_atm(loader, n_clusters=5, allowed_percent=4)

In [6]:
geo_df

,TID,longitude,latitude
0,692835,37.646257,55.742062
1,698656,37.666136,55.731231
2,686168,37.675027,55.727031
3,679671,37.669706,55.706824
4,682180,37.596399,55.711151
...,...,...,...
1625,607749,37.138157,56.757843
1626,699046,35.647877,56.226824
1627,637512,35.515147,56.032402
1628,680602,36.543716,55.380772


In [7]:
for df, name in zip([clust_df_dichotomous, clust_df_kmeans], ['dichotomous', 'kmeans']):
    print(name)
    geoplot_clusters(df.merge(geo_df), LABEL_COL)

dichotomous


kmeans
